In [ ]:
using JLD
include("Gync.jl");
parms, y0 = loadparms()
data = loadpfizer();

In [ ]:
SIGMA_RHO = 0.1
SIGMA_Y0  = 0.1
SIGMA_PARMS = 10
SIGMA_PROPOSAL = 0.05
adapt = :all
stepsize = 100
sampler = :amm
persons = 1:3
samplingnodes = [:sparms, :y0]

jldname = "smp $sampler ad $adapt pr imd-flat per $(length(persons))-r$SIGMA_RHO-sy$SIGMA_Y0-spar$SIGMA_PARMS-spro$SIGMA_PROPOSAL.jld"

d = eye(length(SAMPLEPARMS)+length(y0))
proposalvariance = d * log(1+(SIGMA_PROPOSAL^2));

samplers = Dict(
    :amm=>[AMM(samplingnodes, proposalvariance, adapt=adapt)], 
    :amwg=>[AMWG(samplingnodes,diag(proposalvariance); target=0.44)])

mcs = []
for person in persons
    m, inp, ini = gyncmodel(data[person], parms, y0)
    setsamplers!(m, samplers[sampler])
    
    push!(mcs, mcmc(m, inp, ini, stepsize, burnin=0, verbose = false, thin = 1))
end

#samplelength(mc) = max(floor(Int, size(mc, 1)/5), 2_000)

println(jldname)
while size(mcs[1],1) < 1_000_000
    @time for i in 1:length(persons)
        mcs[persons[i]] = mcmc(mcs[persons[i]], stepsize, verbose=false)
    end
    @time save(jldname, "mc", [mc.value for mc in mcs])    
    println(size(mcs[1]))
end